In [2]:
import import_ipynb
from arrival_networkx import Arrival, get_branch_instance
import numpy as np
import random as rm
from typing import List
from collections import deque
import networkx as nx

importing Jupyter notebook from arrival_networkx.ipynb


In [9]:
def multi_run_procedure(instance: Arrival, S_subset: List[int]):
    s_curr =  instance.s_0
    s_next = instance.s_1
    
    w = {s:1 for s in S_subset}
    
    t = {s:0 for s in instance.vertices}
    t[0] = 1
    for v in S_subset:
        t[instance.s_0[v]] += np.ceil(w[v]/2)
        t[instance.s_1[v]] += np.floor(w[v]/2)
    # print(t)
    
    waiting_set = []
    for v in instance.vertices:
        if v not in S_subset and v not in [instance.target_node, instance.sink_node]:
            waiting_set.append(v)
            
    while len(waiting_set)>0:
        waiting_set_ = [ws for ws in waiting_set if t[ws]>0]
        if not waiting_set_:
            break
        
        choose = rm.choice(waiting_set_)
        # print(choose,t)
        tau = rm.randint(1,t[choose])
        t[choose] -= tau
        t[s_curr[choose]] += np.ceil(tau/2)
        t[s_next[choose]] += np.floor(tau/2)
        
        if tau & 1 : 
            temp = s_curr[choose]
            s_curr[choose] = s_next[choose]
            s_next[choose] = temp
            
        # break
    return t


In [10]:
def decompose_into_layers(instance : Arrival):
    """
    Decompose the graph into layers based on the distance of the vertices to the destination nodes.
    
    Parameters:
    - graph: A NetworkX graph instance.
    - target_node: The target node (d).
    - sink_node: The sink node (d').
    
    Returns:
    - layers: A dictionary where key is the layer index and value is a list of nodes in that layer.
    - max_dist: The maximum distance (layer index) found.
    """
    
    # Initialize the layers dictionary
    layers = {}
    # Distance dictionary with initial values set to None for each node
    dist = {node: float('inf') for node in instance.graph.nodes()}
    
    # Define a BFS procedure to calculate distances to {target_node, sink_node}
    queue = deque([(instance.target_node, 0), (instance.sink_node, 0)])
    while queue:
        current_node, current_dist = queue.popleft()
        if dist[current_node] == float('inf'):
            dist[current_node] = current_dist
            if current_dist not in layers:
                layers[current_dist] = [current_node]
            else:
                layers[current_dist].append(current_node)
            for neighbor in instance.graph.predecessors(current_node):
                if dist[neighbor] == float('inf'):
                    queue.append((neighbor, current_dist + 1))
    
    # Calculate max_dist
    max_dist = max(dist.values())
    
    return layers, max_dist

In [11]:
def compute_phi_set(instance: Arrival, phi: float) :
    layers, max_dist = decompose_into_layers(instance) 
    print(layers)
    S = []
    U = layers[0]
    for i in range(1,len(layers)):
        if len(layers[i]) < phi*len(U):
            S += layers[i]
            U = []
        U += layers[i]
    return S
    

In [12]:
def subexponential(instance: Arrival, phi: float):
    S = compute_phi_set(instance, phi)
    print("S is ",S)
    t = multi_run_procedure(instance, S)
    return t

In [7]:
hard_instance = get_branch_instance(100,0.5)
subexponential(hard_instance, 0.5)

{0: [99, -1], 1: [98, 49], 2: [97, 48], 3: [96, 47], 4: [95, 46], 5: [94, 45], 6: [93, 44], 7: [92, 43], 8: [91, 42], 9: [90, 41], 10: [89, 40], 11: [88, 39], 12: [87, 38], 13: [86, 37], 14: [85, 36], 15: [84, 35], 16: [83, 34], 17: [82, 33], 18: [81, 32], 19: [80, 31], 20: [79, 30], 21: [78, 29], 22: [77, 28], 23: [76, 27], 24: [75, 26], 25: [74, 25], 26: [73, 24], 27: [72, 23], 28: [71, 22], 29: [70, 21], 30: [69, 20], 31: [68, 19], 32: [67, 18], 33: [66, 17], 34: [65, 16], 35: [64, 15], 36: [63, 14], 37: [62, 13], 38: [61, 12], 39: [60, 11], 40: [59, 10], 41: [58, 9], 42: [57, 8], 43: [56, 7], 44: [55, 6], 45: [54, 5], 46: [53, 4], 47: [52, 3], 48: [51, 2], 49: [50, 1], 50: [0]}
S is  [96, 47, 93, 44, 90, 41, 87, 38, 84, 35, 81, 32, 78, 29, 75, 26, 72, 23, 69, 20, 66, 17, 63, 14, 60, 11, 57, 8, 54, 5, 51, 2, 0]
final t array  {-1: 0, 0: 30.0, 1: 0.0, 2: 0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0, 8: 0, 9: 0.0, 10: 0, 11: 0, 12: 0.0, 13: 0, 14: 0, 15: 0.0, 16: 0, 17: 0, 18: 0.0, 19: 0, 

{-1: 0,
 0: 30.0,
 1: 0.0,
 2: 0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0,
 8: 0,
 9: 0.0,
 10: 0,
 11: 0,
 12: 0.0,
 13: 0,
 14: 0,
 15: 0.0,
 16: 0,
 17: 0,
 18: 0.0,
 19: 0,
 20: 0,
 21: 0.0,
 22: 0,
 23: 0,
 24: 0.0,
 25: 0,
 26: 0,
 27: 0.0,
 28: 0.0,
 29: 0,
 30: 0.0,
 31: 0.0,
 32: 0,
 33: 0.0,
 34: 0,
 35: 0,
 36: 0.0,
 37: 0.0,
 38: 0,
 39: 0.0,
 40: 0.0,
 41: 1.0,
 42: 0.0,
 43: 0,
 44: 0,
 45: 0.0,
 46: 0,
 47: 0,
 48: 0.0,
 49: 0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 1.0,
 55: 0.0,
 56: 0,
 57: 0,
 58: 0.0,
 59: 0.0,
 60: 1.0,
 61: 0.0,
 62: 0,
 63: 0,
 64: 0.0,
 65: 0,
 66: 0,
 67: 0.0,
 68: 0.0,
 69: 1.0,
 70: 0.0,
 71: 0,
 72: 0,
 73: 0.0,
 74: 0.0,
 75: 0,
 76: 0.0,
 77: 0,
 78: 0,
 79: 0.0,
 80: 0.0,
 81: 0,
 82: 0.0,
 83: 0,
 84: 0,
 85: 0.0,
 86: 0,
 87: 0,
 88: 0.0,
 89: 0,
 90: 0,
 91: 0.0,
 92: 0,
 93: 0,
 94: 0.0,
 95: 0,
 96: 0,
 97: 0.0,
 98: 0,
 99: 0}

In [13]:
instance = Arrival(100, True)
subexponential(instance, 0.5)

{0: [99, -1], 1: [6, 48, 60, 61, 64], 2: [3, 62, 77, 55, 75, 93, 45, 49, 63, 76, 52, 83, 89], 3: [19, 33, 66, 8, 34, 84, 29, 38, 20, 81, 71, 16, 91, 57], 4: [35, 65, 40, 53, 72, 28, 12, 21, 67, 41, 4, 88], 5: [23, 30, 82, 39, 46, 43, 85, 37, 51, 25, 22, 92], 6: [86, 27, 15, 78, 54, 14, 13, 68, 80, 94, 74, 90, 44, 79, 32, 73], 7: [50, 56, 87, 58, 9, 36, 69, 59, 95, 7, 98, 26, 70, 97, 1], 8: [24, 0, 17, 18, 11, 96, 42, 5, 47, 10], 9: [2, 31]}
S is  [35, 65, 40, 53, 72, 28, 12, 21, 67, 41, 4, 88, 50, 56, 87, 58, 9, 36, 69, 59, 95, 7, 98, 26, 70, 97, 1, 2, 31]


{-1: 0,
 0: 0.0,
 1: 0,
 2: 2.0,
 3: 0.0,
 4: 1.0,
 5: 0.0,
 6: 0.0,
 7: 1.0,
 8: 0.0,
 9: 0.0,
 10: 0,
 11: 0,
 12: 0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0,
 17: 0,
 18: 0,
 19: 0.0,
 20: 0,
 21: 0,
 22: 0.0,
 23: 0,
 24: 0.0,
 25: 0.0,
 26: 0,
 27: 0.0,
 28: 2.0,
 29: 0,
 30: 0.0,
 31: 1.0,
 32: 0.0,
 33: 0,
 34: 0.0,
 35: 0,
 36: 2.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 2.0,
 42: 0.0,
 43: 0,
 44: 0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0,
 50: 2.0,
 51: 0.0,
 52: 0.0,
 53: 1.0,
 54: 0.0,
 55: 0,
 56: 2.0,
 57: 0.0,
 58: 2.0,
 59: 1.0,
 60: 0,
 61: 0,
 62: 0.0,
 63: 0,
 64: 0,
 65: 2.0,
 66: 0.0,
 67: 2.0,
 68: 0.0,
 69: 1.0,
 70: 0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0,
 75: 0.0,
 76: 0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0,
 84: 0.0,
 85: 0,
 86: 0.0,
 87: 0,
 88: 2.0,
 89: 0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0,
 94: 0.0,
 95: 2.0,
 96: 0.0,
 97: 1.0,
 98: 0.0,
 99: 1.0}